# Imports

In [1]:
import json
from ast import literal_eval
import numpy as np
import kernels

c:\Users\felix\miniforge3\envs\qml\Lib\site-packages\gpflow\versions.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


# Load in data

In [ ]:
# open the json file with the quantum kernel calculation results
with open('kernel_qpu_test_results/kernel_cpu_test_results_240829.json') as f:
    data = json.loads(f.read())

num_features = data['kernel_data']['num_features']
num_repetitions = data['kernel_data']['num_repetitions']

xrd = np.array(data['xrd_data']['xrd'])
theta = np.array(data['xrd_data']['theta'])
num_points = xrd.shape[0]

In [ ]:
true_labels = np.array([
    3, 3, 3, 3,
    2, 2, 2, 2,
    4, 4, 4, 4, 
    1, 1, 1, 1,
    0, 0, 0, 0,
])

In [5]:
qpu_kernel_matrix = np.ones((num_points, num_points), dtype=float)

for (k, v) in data['kernel_entries'].items():
    i, j = literal_eval(k)

    try:
        qpu_kernel_matrix[i, j] = v['result']['0']
        qpu_kernel_matrix[j, i] = v['result']['0']
    except KeyError:
        qpu_kernel_matrix[i, j] = 0
        qpu_kernel_matrix[j, i] = 0

# Recreate classical kernel for comparison

In [6]:
cpu_kernel_matrix = kernels.get_cosine_kernel_matrix()

# Calculate $g_{CQ}$

In [ ]:
cU, cS, cVh = np.linalg.svd(cpu_kernel_matrix, hermitian=True)

qU, qS, qVh = np.linalg.svd(qpu_kernel_matrix, hermitian=True)

In [9]:
sqrt_qK = qU @ np.diag(np.sqrt(qS)) @ qVh

In [10]:
inv_cK = cVh.T @ np.diag(cS**(-1)) @ cU.T

In [11]:
tS = np.linalg.svd(sqrt_qK @ inv_cK @ sqrt_qK,
                   compute_uv=False)

In [ ]:
eig_vals, eig_vecs = np.linalg.eig(sqrt_qK @ inv_cK @ sqrt_qK)

# calculate the display the engineered binary labels
y = np.array([0 if x < 0 else 1 for x in sqrt_qK @ eig_vecs[:, 0]])

y

array([1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0])

In [13]:
g_cq = np.sqrt(np.max(tS))

In [14]:
print(g_cq)

11.785615438001624


# Calculate model complexities

In [15]:
inv_qK = qVh.T @ np.diag(qS**(-1)) @ qU.T

In [ ]:
s_C = 0
s_Q = 0

for i in range(inv_cK.shape[0]):
    for j in range(inv_cK.shape[1]):
        s_C += inv_cK[i, j] * (1 if true_labels[i] == true_labels[j] else -1)
        s_Q += inv_qK[i, j] * (1 if true_labels[i] == true_labels[j] else -1)

In [17]:
print(s_C)
print(s_Q)

37.19709107574981
19.43772520120927
